In [1]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import fundamentalanalysis as fa
from datetime import datetime
FA_API_KEY = '85fe259a4ec6fad3cbe55a5ddaf7f9b4'

# save for 3 hours
def load_data(ticker):
    # load data
    profile = fa.profile(ticker, FA_API_KEY)
    key_metrics_annually = fa.key_metrics(ticker, FA_API_KEY, period="annual")
    stock_data = fa.stock_data(ticker, period="5y", interval="1d")
    financial_ratios_annually = fa.financial_ratios(ticker, FA_API_KEY, period="annual")
    income_statement_annually = fa.income_statement(ticker, FA_API_KEY, period="annual")
    try:
        dividends = fa.stock_dividend(ticker, FA_API_KEY)
        dividends.index = pd.to_datetime(dividends.index)
        dividends = dividends["adjDividend"].resample("1Y").sum().sort_index()
    except:
        dividends = pd.Series(0, name="Dividends")

    # return information of interest
    return {
        "stock_closings": stock_data["close"].sort_index(),
        "historical_PE": key_metrics_annually.loc["peRatio"].sort_index(),
        "payout_ratio": financial_ratios_annually.loc["payoutRatio"].sort_index(),
        "dividend_yield": 100*financial_ratios_annually.loc["dividendYield"].sort_index(),
        "cash_per_share": key_metrics_annually.loc["cashPerShare"].sort_index(),
        "debt_to_equity": key_metrics_annually.loc["debtToEquity"].sort_index(),
        "free_cash_flow_per_share": key_metrics_annually.loc["freeCashFlowPerShare"].sort_index(),
        "dividends": dividends,
        "earnings_per_share": income_statement_annually.loc["eps"].sort_index(),
        "info": profile.iloc[:, 0]
    }

In [2]:
stock_data = load_data("AAPL")

In [9]:
info = stock_data["info"]
info

symbol                                                            AAPL
price                                                           152.55
beta                                                          1.277894
volAvg                                                        75166090
mktCap                                                   2413630810829
lastDiv                                                           0.91
range                                                    124.17-179.61
changes                                                          -1.16
companyName                                                 Apple Inc.
currency                                                           USD
cik                                                         0000320193
isin                                                      US0378331005
cusip                                                        037833100
exchange                                          NASDAQ Global Select
exchan

In [12]:
stock_data = fa.stock_data("AAPL", period="5y", interval="1d")
stock_data.head()

,low,close,high,open,volume,adjclose
2018-02-23,42.927502,43.125000,43.487499,42.950001,123967600,41.111969
2018-02-24,43.384998,43.875000,43.912498,43.417500,135249600,41.826958
2018-02-27,44.052502,44.742500,44.847500,44.087502,152648800,42.653973
2018-02-28,44.540001,44.597500,45.119999,44.775002,155712400,42.515732
2018-03-01,44.512501,44.529999,45.154999,44.814999,151128400,42.451385


In [18]:
x = stock_data["close"]
x.index

Index([2018-02-23, 2018-02-24, 2018-02-27, 2018-02-28, 2018-03-01, 2018-03-02,
       2018-03-03, 2018-03-06, 2018-03-07, 2018-03-08,
       ...
       2023-02-08, 2023-02-09, 2023-02-10, 2023-02-11, 2023-02-14, 2023-02-15,
       2023-02-16, 2023-02-17, 2023-02-18, 2023-02-22],
      dtype='object', length=1258)

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import fundamentalanalysis as fa
from datetime import datetime
FA_API_KEY = '85fe259a4ec6fad3cbe55a5ddaf7f9b4'

profile = fa.profile('AAPL', FA_API_KEY)

In [3]:
profile

,0
symbol,AAPL
price,146.71
beta,1.277894
volAvg,74992295
mktCap,2321230916137
lastDiv,0.91
range,124.17-179.61
changes,-2.69
companyName,Apple Inc.
currency,USD


In [4]:
num = 1000000

formatted_num = f'{num:,}'

print(formatted_num)  # Output: 1,000,000

1,000,000
